In [16]:
##import data and libraries

In [17]:
import numpy as np
import pandas as pd
from math import sqrt
from datetime import datetime
from sklearn.metrics import mean_squared_error
import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

df = pd.read_csv("municipality_bus_utilization.csv")
df

,timestamp,municipality_id,usage,total_capacity
0,2017-06-04 07:59:42,9,454,1332
1,2017-06-04 07:59:42,8,556,2947
2,2017-06-04 07:59:42,4,1090,3893
3,2017-06-04 07:59:42,0,204,2813
4,2017-06-04 07:59:42,7,718,2019
...,...,...,...,...
13065,2017-08-19 16:30:35,2,548,697
13066,2017-08-19 16:30:35,8,1193,2947
13067,2017-08-19 16:30:35,7,1354,2019
13068,2017-08-19 16:30:35,6,1680,3113


## Creating necessary dataframe

In [18]:
# convert the timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# extract hour from the timestamp column to create an time_hour column
hour = df['hour'] = df['timestamp'].dt.hour
#minute = df['minute'] = df['timestamp'].dt.minute
time = df['time'] = df['timestamp'].dt.date

print(df['timestamp'])


0       2017-06-04 07:59:42
1       2017-06-04 07:59:42
2       2017-06-04 07:59:42
3       2017-06-04 07:59:42
4       2017-06-04 07:59:42
                ...        
13065   2017-08-19 16:30:35
13066   2017-08-19 16:30:35
13067   2017-08-19 16:30:35
13068   2017-08-19 16:30:35
13069   2017-08-19 16:30:35
Name: timestamp, Length: 13070, dtype: datetime64[ns]


## all data and columns 1(municipaly_id)

In [19]:
municipaly_id = df.iloc[:,1:2].values
municipaly_id = pd.DataFrame(data=municipaly_id, columns = ['municipaly_id'])

print(municipaly_id)

       municipaly_id
0                  9
1                  8
2                  4
3                  0
4                  7
...              ...
13065              2
13066              8
13067              7
13068              6
13069              3

[13070 rows x 1 columns]


## to complete the missing values

In [20]:
others = df.iloc[:,2:4].values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
impdata = others[:,:]
imputer = imputer.fit(impdata[:,:])
impdata[:,:] = imputer.transform(impdata[:,:])
finalothers = impdata[:,:]

finalothers = pd.DataFrame(data=finalothers, columns = ['usage', 'total_capacity'])
print("final others :")
print(finalothers)

data = pd.concat([df['timestamp'], municipaly_id], axis=1)
print("with timestap :")
print(data)
##Concatenating 
data = pd.concat([data, finalothers], axis=1)
print("Concatenating data:")
print(data)
## create train and test data
traindata = data.iloc[:10390,:]
testdata = data.iloc[10390:,:]
#sort data
traindata = traindata.sort_values(["municipaly_id","timestamp"], ascending = (True,True))
testdata = testdata.sort_values(["municipaly_id","timestamp"], ascending = (True,True))
print("train data :")
print(traindata)

final others :
       usage  total_capacity
0        454            1332
1        556            2947
2       1090            3893
3        204            2813
4        718            2019
...      ...             ...
13065    548             697
13066   1193            2947
13067   1354            2019
13068   1680            3113
13069   1189            1930

[13070 rows x 2 columns]
with timestap :
                timestamp  municipaly_id
0     2017-06-04 07:59:42              9
1     2017-06-04 07:59:42              8
2     2017-06-04 07:59:42              4
3     2017-06-04 07:59:42              0
4     2017-06-04 07:59:42              7
...                   ...            ...
13065 2017-08-19 16:30:35              2
13066 2017-08-19 16:30:35              8
13067 2017-08-19 16:30:35              7
13068 2017-08-19 16:30:35              6
13069 2017-08-19 16:30:35              3

[13070 rows x 2 columns]
Concatenating data:
                timestamp  municipaly_id  usage  total_ca

In [21]:

traindata0 = traindata.iloc[:1039,:]
traindata1 = traindata.iloc[1039:2078,:]
traindata2 = traindata.iloc[2078:3117,:]
traindata3 = traindata.iloc[3117:4156,:]
traindata4 = traindata.iloc[4156:5195,:]
traindata5 = traindata.iloc[5195:6234,:]
traindata6 = traindata.iloc[6234:7273,:]
traindata7 = traindata.iloc[7273:8312,:]
traindata8 = traindata.iloc[8312:9351,:]
traindata9 = traindata.iloc[9351:,:]

print(traindata0)

print(traindata1)

print(traindata2)

print(traindata3)

print(traindata4)

print(traindata5)

print(traindata6)

print(traindata7)

print(traindata8)

print(traindata9)


print(testdata)

                timestamp  municipaly_id  usage  total_capacity
3     2017-06-04 07:59:42              0    204            2813
18    2017-06-04 08:25:42              0    247            2813
29    2017-06-04 08:59:42              0    332            2813
34    2017-06-04 09:32:46              0    429            2813
47    2017-06-04 09:59:48              0    485            2813
...                   ...            ...    ...             ...
10342 2017-08-02 14:25:50              0    656            2813
10355 2017-08-02 14:58:50              0    646            2813
10361 2017-08-02 15:31:52              0    635            2813
10370 2017-08-02 15:58:52              0    616            2813
10385 2017-08-02 16:32:54              0    638            2813

[1039 rows x 4 columns]
                timestamp  municipaly_id  usage  total_capacity
7     2017-06-04 07:59:42              1    129             397
19    2017-06-04 08:25:42              1    138             397
24    2017-06-0

In [22]:

testdata0 = testdata.iloc[:268,:]
testdata1 = testdata.iloc[268:536,:]
testdata2 = testdata.iloc[536:804,:]
testdata3 = testdata.iloc[804:1072,:]
testdata4 = testdata.iloc[1072:1340,:]
testdata5 = testdata.iloc[1340:1608,:]
testdata6 = testdata.iloc[1608:1876,:]
testdata7 = testdata.iloc[1876:2144,:]
testdata8 = testdata.iloc[2144:2412,:]
testdata9 = testdata.iloc[2412:,:]

print(testdata0)

print(testdata1)

print(testdata2)

print(testdata3)

print(testdata4)

print(testdata5)

print(testdata6)

print(testdata7)

print(testdata8)

print(testdata9)

                timestamp  municipaly_id  usage  total_capacity
10398 2017-08-05 08:02:03              0    209            2813
10407 2017-08-05 08:29:08              0    252            2813
10416 2017-08-05 09:02:08              0    322            2813
10425 2017-08-05 09:29:08              0    393            2813
10434 2017-08-05 10:03:04              0    460            2813
...                   ...            ...    ...             ...
13024 2017-08-19 14:30:33              0   1285            2813
13031 2017-08-19 15:03:34              0   1283            2813
13041 2017-08-19 15:29:33              0   1286            2813
13059 2017-08-19 16:03:35              0   1276            2813
13062 2017-08-19 16:30:35              0   1271            2813

[268 rows x 4 columns]
                timestamp  municipaly_id  usage  total_capacity
10390 2017-08-05 08:02:03              1    141             397
10400 2017-08-05 08:29:08              1    158             397
10410 2017-08-05

#### 

In [23]:
finaldata = traindata.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

finaldata0 = traindata0.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata0 = traindata0.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata0)

finaldata1 = traindata1.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata1 = traindata1.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata1)

finaldata2 = traindata2.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata2 = traindata2.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata2)

finaldata3 = traindata3.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata3 = traindata3.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata3)

finaldata4 = traindata4.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata4 = traindata4.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata4)

finaldata5 = traindata5.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata5 = traindata5.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata5)

finaldata6 = traindata6.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata6 = traindata6.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata6)

finaldata7 = traindata7.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata7 = traindata7.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata7)

finaldata8 = traindata8.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata8 = traindata8.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata8)

finaldata9 = traindata9.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tsdata9 = traindata9.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata9)

print(finaldata)

print(finaldata0)

print(finaldata1)

print(finaldata2)

print(finaldata3)

print(finaldata4)

print(finaldata5)

print(finaldata6)

print(finaldata7)

print(finaldata8)

print(finaldata9)

timestamp
2017-06-04 07:00:00    204.0
2017-06-04 08:00:00    332.0
2017-06-04 09:00:00    485.0
2017-06-04 10:00:00    583.0
2017-06-04 11:00:00    614.0
                       ...  
2017-08-02 12:00:00    661.0
2017-08-02 13:00:00    667.0
2017-08-02 14:00:00    656.0
2017-08-02 15:00:00    635.0
2017-08-02 16:00:00    638.0
Freq: H, Name: usage, Length: 1426, dtype: float64
timestamp
2017-06-04 07:00:00    129.0
2017-06-04 08:00:00    164.0
2017-06-04 09:00:00    245.0
2017-06-04 10:00:00    317.0
2017-06-04 11:00:00    341.0
                       ...  
2017-08-02 12:00:00    411.0
2017-08-02 13:00:00    411.0
2017-08-02 14:00:00    406.0
2017-08-02 15:00:00    391.0
2017-08-02 16:00:00    358.0
Freq: H, Name: usage, Length: 1426, dtype: float64
timestamp
2017-06-04 07:00:00    273.0
2017-06-04 08:00:00    355.0
2017-06-04 09:00:00    494.0
2017-06-04 10:00:00    582.0
2017-06-04 11:00:00    643.0
                       ...  
2017-08-02 12:00:00    683.0
2017-08-02 13:00:00    694.

In [24]:

trydata = testdata.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

trydata0 = testdata0.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata0 = testdata0.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(trydata0)

trydata1 = testdata1.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata1 = testdata1.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata1)

trydata2 = testdata2.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata2 = testdata2.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata2)

trydata3 = testdata3.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata3 = testdata3.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata3)

trydata4 = testdata4.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata4 = testdata4.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata4)

trydata5 = testdata5.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata5 = testdata5.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata5)

trydata6 = testdata6.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata6 = testdata6.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata6)

trydata7 = testdata7.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata7 = testdata7.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata7)

trydata8 = testdata8.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata8 = testdata8.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata8)

trydata9 = testdata9.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata9 = testdata9.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata9)

print(trydata)

print(trydata0)

print(trydata1)

print(trydata2)

print(trydata3)

print(trydata4)

print(trydata5)

print(trydata6)

print(trydata7)

print(trydata8)

print(trydata9)

     municipaly_id  usage
0                0    252
1                0    393
2                0    517
3                0    653
4                0    655
..             ...    ...
137              0   1281
138              0   1283
139              0   1285
140              0   1286
141              0   1276

[142 rows x 2 columns]
timestamp
2017-08-05 08:00:00    158.0
2017-08-05 09:00:00    216.0
2017-08-05 10:00:00    300.0
2017-08-05 11:00:00    397.0
2017-08-05 12:00:00    398.0
                       ...  
2017-08-19 12:00:00    406.0
2017-08-19 13:00:00    408.0
2017-08-19 14:00:00    410.0
2017-08-19 15:00:00    411.0
2017-08-19 16:00:00    401.0
Freq: H, Name: usage, Length: 345, dtype: float64
timestamp
2017-08-05 08:00:00    327.0
2017-08-05 09:00:00    405.0
2017-08-05 10:00:00    483.0
2017-08-05 11:00:00    567.0
2017-08-05 12:00:00    584.0
                       ...  
2017-08-19 12:00:00    663.0
2017-08-19 13:00:00    697.0
2017-08-19 14:00:00    687.0
2017-08-19 15:

## Forecasting

In [25]:
warnings.filterwarnings("ignore")

# Predict0
model0 = ExponentialSmoothing(finaldata0["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model0 = model0.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred0 = hw_model0.predict(start=trydata0["usage"].index[0], end=trydata0["usage"].index[-1])

pred0.index += len(finaldata0)

trydata0.index += len(finaldata0)

print(pred0)


# Predict1
model1 = ExponentialSmoothing(finaldata1["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model1 = model1.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred1 = hw_model1.predict(start=trydata1["usage"].index[0], end=trydata1["usage"].index[-1])

pred1.index += len(finaldata1)

trydata1.index += len(finaldata1)

print(pred1)


# Predict2
model2 = ExponentialSmoothing(finaldata2["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model2 = model2.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred2 = hw_model2.predict(start=trydata2["usage"].index[0], end=trydata2["usage"].index[-1])

pred2.index += len(finaldata2)

trydata2.index += len(finaldata2)

print(pred2)


# Predict3
model3 = ExponentialSmoothing(finaldata3["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model3 = model3.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred3 = hw_model3.predict(start=trydata3["usage"].index[0], end=trydata3["usage"].index[-1])

pred3.index += len(finaldata3)

trydata3.index += len(finaldata3)

print(pred3)


# Predict4
model4 = ExponentialSmoothing(finaldata4["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model4 = model4.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred4 = hw_model4.predict(start=trydata4["usage"].index[0], end=trydata4["usage"].index[-1])

pred4.index += len(finaldata4)

trydata4.index += len(finaldata4)

print(pred4)

# Predict5
model5 = ExponentialSmoothing(finaldata5["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model5 = model5.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred5 = hw_model5.predict(start=trydata5["usage"].index[0], end=trydata5["usage"].index[-1])

pred5.index += len(finaldata5)

trydata5.index += len(finaldata5)

print(pred5)

# Predict6
model6 = ExponentialSmoothing(finaldata6["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model6 = model6.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred6 = hw_model6.predict(start=trydata6["usage"].index[0], end=trydata6["usage"].index[-1])

pred6.index += len(finaldata6)

trydata6.index += len(finaldata6)

print(pred6)


model7 = ExponentialSmoothing(finaldata7["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model7 = model7.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred7 = hw_model7.predict(start=trydata7["usage"].index[0], end=trydata7["usage"].index[-1])

pred7.index += len(finaldata7)

trydata7.index += len(finaldata7)

print(pred7)

# Predict8
model8 = ExponentialSmoothing(finaldata8["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model8 = model8.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred8 = hw_model8.predict(start=trydata8["usage"].index[0], end=trydata8["usage"].index[-1])

pred8.index += len(finaldata8)

trydata8.index += len(finaldata8)

print(pred8)

# Predict9
model9 = ExponentialSmoothing(finaldata9["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model9 = model9.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred9 = hw_model9.predict(start=trydata9["usage"].index[0], end=trydata9["usage"].index[-1])

pred9.index += len(finaldata9)

trydata9.index += len(finaldata9)

print(pred9)


532    358.600000
533    204.773000
534    331.363865
535    484.231819
536    582.506159
          ...    
669    696.949041
670    691.029745
671    679.060149
672    630.245301
673    548.411227
Length: 142, dtype: float64
532     73.931250
533    128.999999
534    163.999999
535    244.999999
536    316.999999
          ...    
669    349.000000
670    341.000000
671    327.000000
672    266.000001
673    215.000001
Length: 142, dtype: float64
532    380.666667
533    273.538333
534    354.592692
535    493.302963
536    581.556515
          ...    
669    624.899545
670    594.154498
671    563.155772
672    477.430779
673    399.392154
Length: 142, dtype: float64
532     963.733333
533     624.703667
534     965.288518
535    1261.511443
536    1395.327557
          ...     
669    1417.999018
670    1381.184995
671    1338.215925
672    1173.826080
673     966.044130
Length: 142, dtype: float64
532    2021.800000
533    1094.659000
534    2011.393295
535    2807.001966
536    31